<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Libaries" data-toc-modified-id="Libaries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libaries</a></span></li><li><span><a href="#Connect-to-Database" data-toc-modified-id="Connect-to-Database-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Connect to Database</a></span></li><li><span><a href="#Load-to-Dataframe" data-toc-modified-id="Load-to-Dataframe-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load to Dataframe</a></span></li></ul></li><li><span><a href="#EDA---Daily-weather-data" data-toc-modified-id="EDA---Daily-weather-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA - Daily weather data</a></span></li><li><span><a href="#EDA---Hourly-weather-data" data-toc-modified-id="EDA---Hourly-weather-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA - Hourly weather data</a></span><ul class="toc-item"><li><span><a href="#Details-Station-Rosengartenstrasse" data-toc-modified-id="Details-Station-Rosengartenstrasse-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Details Station Rosengartenstrasse</a></span></li><li><span><a href="#Details-Station-Stampfenbachstrasse" data-toc-modified-id="Details-Station-Stampfenbachstrasse-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Details Station Stampfenbachstrasse</a></span></li><li><span><a href="#Details-Station-Schimmelstrasse" data-toc-modified-id="Details-Station-Schimmelstrasse-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Details Station Schimmelstrasse</a></span></li></ul></li></ul></div>

In [98]:
# dependencies
import pandas as pd
import psycopg2 as pg
from datetime import date
from sqlalchemy import create_engine, text, inspect
import requests
import json
import datetime
import logging
import matplotlib.pyplot as plt

## Preparation

### Libaries

In [12]:
import pandas as pd
import psycopg2 as pg
from datetime import date
from sqlalchemy import create_engine, text, inspect
import requests
import json
import datetime
import logging

### Connect to Database

In [3]:
# DB credentials
ENDPOINT = 'trunks.cwh7dj4q8zyj.us-east-1.rds.amazonaws.com'
DB_NAME = 'trunks'
USERNAME = 'trunks'
PASSWORD = 'Corn5!!!'


# connection string for sql alchemy engine
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DB_NAME}'

In [9]:
# create sql alchemy engine
db = create_engine(conn_string)
dbconnect = db.connect()

In [7]:
 # check existing tables in DB
insp = inspect(db)
existing_tables = insp.get_table_names()
print(existing_tables)

['weather_hourly_urls', 'weather_daily', 'weather_daily_urls', 'weather_hourly']


### Load to Dataframe

We first load both tables (daily and houly) into pandas dataframes 

In [11]:
df_daily = pd.read_sql_table('weather_daily', dbconnect) #daily aggregated data
df_hourly = pd.read_sql_table('weather_hourly', dbconnect) #hourly data (note: longer loading time)

## EDA - Daily weather data

In [97]:
df_daily = df_daily.sort_values("Standort",ascending= False)
df_daily["Wert"] = pd.to_numeric(df_daily["Wert"], downcast="float")
df_daily = df_daily.sort_values("Datum",ascending= False)
df_daily.head(15)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
67643,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,T,d1,°C,12.370000,provisorisch
67654,2022-04-20T00:00+0100,Zch_Rosengartenstrasse,p,d1,hPa,960.000000,provisorisch
67653,2022-04-20T00:00+0100,Zch_Rosengartenstrasse,RainDur,d1,min,0.000000,provisorisch
67655,2022-04-20T00:00+0100,Zch_Rosengartenstrasse,T_max_h1,d1,°C,16.440001,provisorisch
67652,2022-04-20T00:00+0100,Zch_Rosengartenstrasse,T,d1,°C,12.010000,provisorisch
67647,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,T_max_h1,d1,°C,16.379999,provisorisch
67645,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,p,d1,hPa,959.789978,provisorisch
67648,2022-04-20T00:00+0100,Zch_Schimmelstrasse,T,d1,°C,12.420000,provisorisch
67646,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,StrGlo,d1,W/m2,296.579987,provisorisch
67644,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,RainDur,d1,min,0.000000,provisorisch


In [46]:
len(df_daily)

86610

In [32]:
df_daily["Standort"].unique()

array(['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse',
       'Zch_Rosengartenstrasse'], dtype=object)

We see that we have in total 3 unique values for each "Standort". That also matches with our buiness understanding.

In [34]:
df_daily["Parameter"].unique()

array(['T', 'RainDur', 'p', 'StrGlo', 'T_max_h1'], dtype=object)

Wee see here that we have for each day and each Station 5 unique messurements. In the daily aggrgated data we have besides the average temperature also the maximal temperature called "T_max_h1".

In [35]:
#df_daily.info()

In [36]:
df_daily.describe()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,86610,86610,86610,86610,86610,85515,86610
unique,10883,3,5,1,4,20319,2
top,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,T,d1,°C,0,bereinigt
freq,13,50742,20302,86610,40604,6715,75716


In [23]:
df_daily.sort_values("Datum")

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
46497,1992-07-01T00:00+0100,Zch_Stampfenbachstrasse,StrGlo,d1,W/m2,135.06,bereinigt
46498,1992-07-01T00:00+0100,Zch_Stampfenbachstrasse,T,d1,°C,18.99,bereinigt
46499,1992-07-01T00:00+0100,Zch_Stampfenbachstrasse,p,d1,hPa,963.06,bereinigt
46500,1992-07-01T00:00+0100,Zch_Stampfenbachstrasse,T_max_h1,d1,°C,21.72,bereinigt
46504,1992-07-02T00:00+0100,Zch_Stampfenbachstrasse,T_max_h1,d1,°C,21.98,bereinigt
...,...,...,...,...,...,...,...
67646,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,StrGlo,d1,W/m2,296.58,provisorisch
67645,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,p,d1,hPa,959.79,provisorisch
67644,2022-04-20T00:00+0100,Zch_Stampfenbachstrasse,RainDur,d1,min,0,provisorisch
67650,2022-04-20T00:00+0100,Zch_Schimmelstrasse,p,d1,hPa,962.37,provisorisch


We can see that the first records were made in July 1992 and the most current records reach to the date of the last run of the script in 2022.

## EDA - Hourly weather data

In [68]:
df_hourly = df_hourly.sort_values("Standort",ascending= False)
df_hourly["Wert"] = pd.to_numeric(df_hourly["Wert"], downcast="float")
df_hourly = df_hourly.sort_values("Datum",ascending= False)
df_hourly.head(15)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
2549681,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,WVs,h1,m/s,0.94,provisorisch
2549677,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,p,h1,hPa,956.57,provisorisch
2549662,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,p,h1,hPa,956.42,provisorisch
2549663,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,RainDur,h1,min,0,provisorisch
2549667,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,WVs,h1,m/s,3.32,provisorisch
2549664,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,StrGlo,h1,W/m2,844.62,provisorisch
2549660,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,T,h1,°C,14.41,provisorisch
2549661,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,Hr,h1,%Hr,37.87,provisorisch
2549673,2022-04-21T11:00+0100,Zch_Schimmelstrasse,WVv,h1,m/s,1.09,provisorisch
2549675,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,T,h1,°C,13.75,provisorisch


In [59]:
df_hourly["Standort"].unique()

array(['Zch_Rosengartenstrasse', 'Zch_Stampfenbachstrasse',
       'Zch_Schimmelstrasse'], dtype=object)

In [45]:
len(df_hourly)

3297717

In [91]:
df_hourly_temperature = df_hourly[df_hourly["Parameter"] == "T"]

In [95]:
min(df_hourly_temperature["Wert"])

-13.25

In [96]:
max(df_hourly_temperature["Wert"])

36.54999923706055

The lowest temperature messured in one of the 3 stations was around -13°C and the highest was around 36°C. This messurements seems reasonable for the city.

Now let quickly check if the timesseries on the differnet stations is equal

In [56]:
len(df_hourly[df_hourly["Standort"] == "Zch_Rosengartenstrasse"])

475052

In [57]:
len(df_hourly[df_hourly["Standort"] == "Zch_Stampfenbachstrasse"])

1983949

In [58]:
len(df_hourly[df_hourly["Standort"] == "Zch_Schimmelstrasse"])

838716

Oh, we can see that there are some very big differences. Lets have a closer look at each station.

### Details Station Rosengartenstrasse

In [65]:
df_hourly_Rosengarten = df_hourly[df_hourly["Standort"] == "Zch_Rosengartenstrasse"]
df_hourly_Rosengarten.sort_values("Datum",ascending= False)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
2549681,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,WVs,h1,m/s,0.94,provisorisch
2549678,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,RainDur,h1,min,0,provisorisch
2549679,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,WD,h1,°,115.16,provisorisch
2549680,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,WVv,h1,m/s,0.52,provisorisch
2549675,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,T,h1,°C,13.75,provisorisch
...,...,...,...,...,...,...,...
777054,2013-10-03T16:00+0100,Zch_Rosengartenstrasse,p,h1,hPa,969.2,bereinigt
777037,2013-10-03T15:00+0100,Zch_Rosengartenstrasse,RainDur,h1,min,0,bereinigt
777038,2013-10-03T15:00+0100,Zch_Rosengartenstrasse,p,h1,hPa,969.53,bereinigt
777022,2013-10-03T14:00+0100,Zch_Rosengartenstrasse,p,h1,hPa,968.99,bereinigt


In [72]:
df_hourly_Rosengarten.describe()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,475052,475052,475052,475052,475052,466242,475052
unique,74923,1,7,1,6,29677,2
top,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,p,h1,m/s,0,bereinigt
freq,7,475052,74923,475052,115017,59935,333701


In [73]:
df_hourly_Rosengarten["Parameter"].unique()

array(['WVs', 'T', 'Hr', 'p', 'RainDur', 'WVv', 'WD'], dtype=object)

### Details Station Stampfenbachstrasse

In [64]:
df_hourly_Stampfenbach = df_hourly[df_hourly["Standort"] == "Zch_Stampfenbachstrasse"]
df_hourly_Stampfenbach.sort_values("Datum",ascending= False)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
2549660,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,T,h1,°C,14.41,provisorisch
2549662,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,p,h1,hPa,956.42,provisorisch
2549663,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,RainDur,h1,min,0,provisorisch
2549664,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,StrGlo,h1,W/m2,844.62,provisorisch
2549665,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,WD,h1,°,30.27,provisorisch
...,...,...,...,...,...,...,...
2447323,1992-01-01T01:00+0100,Zch_Stampfenbachstrasse,WVs,h1,m/s,0.65,bereinigt
2447321,1992-01-01T01:00+0100,Zch_Stampfenbachstrasse,WD,h1,°,317.65,bereinigt
2447320,1992-01-01T00:00+0100,Zch_Stampfenbachstrasse,WVs,h1,m/s,1.24,bereinigt
2447319,1992-01-01T00:00+0100,Zch_Stampfenbachstrasse,WVv,h1,m/s,1.24,bereinigt


In [74]:
df_hourly_Stampfenbach.describe()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,1983949,1983949,1983949,1983949,1983949,1963684,1983949
unique,265641,1,8,1,7,63600,2
top,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,T,h1,m/s,0,bereinigt
freq,8,1983949,261274,1983949,508382,139108,1822405


In [75]:
df_hourly_Stampfenbach["Parameter"].unique()

array(['T', 'Hr', 'p', 'RainDur', 'StrGlo', 'WD', 'WVv', 'WVs'],
      dtype=object)

### Details Station Schimmelstrasse

In [69]:
df_hourly_Schimmel = df_hourly[df_hourly["Standort"] == "Zch_Schimmelstrasse"]
df_hourly_Schimmel.sort_values("Datum",ascending= False)

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,838716,838716,838716,838716,838716,822988,838716
unique,151580,1,7,1,6,34810,2
top,2022-04-21T11:00+0100,Zch_Schimmelstrasse,RainDur,h1,min,0,bereinigt
freq,7,838716,151580,838716,151580,110946,706148


In [70]:
df_hourly_Schimmel.describe()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,838716,838716,838716,838716,838716,822988,838716
unique,151580,1,7,1,6,34810,2
top,2022-04-21T11:00+0100,Zch_Schimmelstrasse,RainDur,h1,min,0,bereinigt
freq,7,838716,151580,838716,151580,110946,706148


In [71]:
df_hourly_Schimmel["Parameter"].unique()

array(['WVv', 'Hr', 'WVs', 'RainDur', 'p', 'WD', 'T'], dtype=object)

Wee see that we have here the same Stantions ("Standorte") like in the daily dataset.

In [41]:
df_hourly["Parameter"].unique()

array(['WVs', 'T', 'Hr', 'p', 'RainDur', 'StrGlo', 'WD', 'WVv'],
      dtype=object)

We see that we have more parameters in the hourly data compated to the daily aggregated ones. New are the windspeed and winddirection. Most importat for uns is the temperature (T) and the Rain duration (RainDur) which we have in both granularities.

In [47]:
df_hourly.describe()

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
count,3297717,3297717,3297717,3297717,3297717,3252914,3297717
unique,265641,3,8,1,7,64296,2
top,2022-04-21T11:00+0100,Zch_Stampfenbachstrasse,T,h1,m/s,0,bereinigt
freq,22,1983949,487631,3297717,767205,309989,2862254


In [42]:
df_hourly.sort_values("Datum")

,Datum,Standort,Parameter,Intervall,Einheit,Wert,Status
2447318,1992-01-01T00:00+0100,Zch_Stampfenbachstrasse,WD,h1,°,343.79,bereinigt
2447319,1992-01-01T00:00+0100,Zch_Stampfenbachstrasse,WVv,h1,m/s,1.24,bereinigt
2447320,1992-01-01T00:00+0100,Zch_Stampfenbachstrasse,WVs,h1,m/s,1.24,bereinigt
2447322,1992-01-01T01:00+0100,Zch_Stampfenbachstrasse,WVv,h1,m/s,0.63,bereinigt
2447321,1992-01-01T01:00+0100,Zch_Stampfenbachstrasse,WD,h1,°,317.65,bereinigt
...,...,...,...,...,...,...,...
2549671,2022-04-21T11:00+0100,Zch_Schimmelstrasse,RainDur,h1,min,0,provisorisch
2549668,2022-04-21T11:00+0100,Zch_Schimmelstrasse,T,h1,°C,14.72,provisorisch
2549673,2022-04-21T11:00+0100,Zch_Schimmelstrasse,WVv,h1,m/s,1.09,provisorisch
2549677,2022-04-21T11:00+0100,Zch_Rosengartenstrasse,p,h1,hPa,956.57,provisorisch


The houly Data start 6 month earlier (01.01.1992) compared to the daily aggregated data. This is no problem for our project becuase for our analysis we are mainly interested in the last 10 year.